In [1]:
# query Terrascope archive using terracatalgoueclient
from terracatalogueclient import Catalogue
import datetime as dt

# processes only VV

# define name of result
name = "testRun_01"

catalogue = Catalogue()

In [2]:
# give one date, find closest acqusition before and after
start = dt.date(2021, 10, 1) # input
end = dt.date(2021, 10, 2)

print("[CATALOGUE]: starting search for scenes...")

# first search
cat = catalogue.get_products(
	"urn:eop:VITO:CGS_S1_SLC_L1",
	start = start,
	end = end
)

for prod in cat:
    print(prod.geometry)

[CATALOGUE]: starting search for scenes...
POLYGON ((7.605903 51.141376, 8.148488 52.759369, 4.297474 53.172043, 3.894983 51.550926, 7.605903 51.141376))
POLYGON ((7.129499 49.65831, 7.649 51.277603, 3.927435 51.687271, 3.53682 50.065575, 7.129499 49.65831))
POLYGON ((6.671979 48.173637, 7.171072 49.794495, 3.568414 50.201855, 3.188162 48.579235, 6.671979 48.173637))


In [76]:
import glob
from pyroSAR import identify
import stsa
import pandas as pd
import sys, os
from contextlib import contextmanager

@contextmanager
def suppress_stdout():
    # source http://thesmithfam.org/blog/2012/10/25/temporarily-suppress-console-output-in-python/
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [135]:
file_list = glob.glob("/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/10/05/*/*.zip")

# load master dataframe
scene_proc = pd.read_pickle("scene_processing.pkl")
# print(scene_proc)

# make list to add to dataframe
meta_list = []

for e in file_list:
    name = e[e.rfind("/")+1:len(e)-4] # extract product name from path
    if not name in scene_proc["name"].values:
        # print("found new scene")
        # use pyroSAR to extract metadata
        pyro = identify(e)

        # make a dict
        meta = {"name": name, "path": e, "sensor": pyro.sensor, 
                "polarizations": pyro.polarizations, "start": pyro.start, "stop": pyro.stop, 
                "mode": pyro.acquisition_mode, "product": pyro.product,
                "orbit_direction": pyro.orbit, "rel_orbit": pyro.orbitNumber_rel}

        # use stsa to create swath geometry
        stsa_geom = stsa.TopsSplitAnalyzer(target_subswaths=['iw1', 'iw2', 'iw3'])
        with suppress_stdout():
            stsa_geom.load_data(zip_path = e)
            stsa_geom._create_subswath_geometry()
        # check whether the scene has a regular burst pattern (no missing, no additional bursts)
        if stsa_geom.df.loc[:,"burst"].to_list() == [1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]:
            meta["regular_burst_pattern"] = 1
        else:
            meta["regular_burst_pattern"] = 0

        # add swath/burst geometry to metadata
        meta["geometry"] = stsa_geom.df
        # append to list
        meta_list.append(meta)
        
        for i, row in scene_proc.iterrows():
            print(row["geometry"])
            # TODO calc overlap etc.
        
    else:
        pass
        # print("old scene")
    
    
    
# print(set(pd.DataFrame(meta_list)["rel_orbit"].to_list()))
# orbits of the year 2021 are {1, 161, 37, 101, 8, 137, 139, 110, 88, 59}
meta_df = pd.DataFrame(meta_list)
# scene_proc = scene_proc.append(meta_df)
scene_proc.to_pickle("scene_processing.pkl")

  subswath burst                                           geometry
0      IW1     1  POLYGON ((7.642016177914016 51.27848259773416,...
1      IW1     2  POLYGON ((7.586869751038266 51.11399463638195,...
2      IW1     3  POLYGON ((7.537012033051871 50.9485961701415, ...
3      IW1     4  POLYGON ((7.48486145628325 50.78388547038926, ...
4      IW1     5  POLYGON ((7.430337822550185 50.61923556501157,...
5      IW1     6  POLYGON ((7.378762990670935 50.45422438230421,...
6      IW1     7  POLYGON ((7.324360005379983 50.28958886496289,...
7      IW1     8  POLYGON ((7.271912790801308 50.12471757205297,...
8      IW1     9  POLYGON ((7.217394900242595 49.96012651850965,...
0      IW2     1  POLYGON ((6.428360639545576 51.3675553628383, ...
1      IW2     2  POLYGON ((6.378401458185031 51.20266526217042,...
2      IW2     3  POLYGON ((6.32889289471817 51.03786132690936, ...
3      IW2     4  POLYGON ((6.278962875828976 50.87299005222759,...
4      IW2     5  POLYGON ((6.226766005699282 50

In [117]:
scene_proc.to_dict()

{'name': {0: 'S1A_IW_SLC__1SDV_20211002T023211_20211002T023238_039934_04B9CC_2515',
  1: 'S1A_IW_SLC__1SDV_20211002T054244_20211002T054311_039936_04B9DC_1F6C',
  2: 'S1A_IW_SLC__1SDV_20211002T054219_20211002T054247_039936_04B9DC_FF3B',
  3: 'S1A_IW_SLC__1SDV_20211002T054309_20211002T054337_039936_04B9DC_7E94',
  4: 'S1B_IW_SLC__1SDV_20211002T174039_20211002T174106_028960_0374AC_601D',
  5: 'S1B_IW_SLC__1SDV_20211002T174014_20211002T174041_028960_0374AC_3B4B',
  6: 'S1B_IW_SLC__1SDV_20211002T174103_20211002T174130_028960_0374AC_7ABF'},
 'path': {0: '/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/10/02/S1A_IW_SLC__1SDV_20211002T023211_20211002T023238_039934_04B9CC_2515/S1A_IW_SLC__1SDV_20211002T023211_20211002T023238_039934_04B9CC_2515.zip',
  1: '/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/10/02/S1A_IW_SLC__1SDV_20211002T054244_20211002T054311_039936_04B9DC_1F6C/S1A_IW_SLC__1SDV_20211002T054244_20211002T054311_039936_04B9DC_1F6C.zip',
  2: '/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/10/02/S1

In [105]:
list = pd.DataFrame([{"hi": 4, "hel": "hi"}, {"hi": 5, "sd": "hellooo"}])
print(list)
print(list.loc[:,"hi"])
print(5 in list.values)

   hi  hel       sd
0   4   hi      NaN
1   5  NaN  hellooo
0    4
1    5
Name: hi, dtype: int64
True


In [75]:

            
print("hi")
with suppress_stdout():
    print("hio")

hi
